# Advanced SQL:

| # | **Tool**                                                  | **Question**                                                       | **When to Use**                                                          |
| - | --------------------------------------------------------- | ------------------------------------------------------------------ | ------------------------------------------------------------------------ |
| 1 | WINDOW FUNCTIONS (`ROW_NUMBER()`, `RANK()`, `AVG() OVER`) | Rank sellers by revenue, calculate moving average of sales         | When doing row-by-row comparisons or rankings within partitions of data  |
| 2 | CTEs (`WITH` clause)                                      | Calculate year-over-year revenue growth with clarity               | When breaking down complex queries into readable, modular steps          |
| 3 | `LAG()` / `LEAD()`                                        | Compare customer’s current vs. last order                          | When comparing values from previous or next rows                         |
| 4 | `DATEDIFF()` + `MIN()`                                    | Find customers who ordered again within 6 months of first purchase | When analyzing time gaps between first and subsequent activities         |
| 5 | `UNION`, `EXCEPT`, `INTERSECT`                            | Identify repeat customers across years                             | When combining, filtering, or intersecting results from multiple queries |
| 6 | Recursive CTEs                                            | Customer referral chain (if applicable)                            | When dealing with hierarchical or recursive data relationships           |


# Use a CTE (Common Table Expression) when you want to:

| **When to Use CTE**           | **Purpose**                                                          |
| ----------------------------- | -------------------------------------------------------------------- |
| ✅ Break down complex logic    | Make queries more readable and maintainable by separating steps.     |
| ✅ Reuse logic                 | Use results (like totals, ranks) multiple times in a bigger query.   |
| ✅ Avoid subquery repetition   | Instead of repeating the same subquery, write it once as a CTE.      |
| ✅ Chain multiple calculations | Perform step-by-step calculations (e.g., aggregate → filter → rank). |
| ✅ Enable recursive operations | Handle hierarchical or tree-like data (e.g., referral chains).       |

Think of a CTE as a temporary named result table that's only visible to the query that follows.


#  WINDOW FUNCTIONS: Rank sellers by revenue.

✅ Goal: Rank sellers by total revenue using RANK() or ROW_NUMBER()

In [1]:
import pandas as pd
from sqlalchemy import create_engine

# SQL Server connection
engine = create_engine(
    "mssql+pyodbc://NARENDRA\\SQLEXPRESS/sql_to_python?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

# Rank sellers by revenue using WINDOW function
query = """
SELECT 
    s.seller_id,
    s.seller_city,
    SUM(oi.price) AS total_revenue,
    RANK() OVER (ORDER BY SUM(oi.price) DESC) AS revenue_rank
FROM order_items oi
JOIN sellers s ON oi.seller_id = s.seller_id
GROUP BY s.seller_id, s.seller_city
ORDER BY revenue_rank;
"""

df = pd.read_sql(query, engine)
df.head()

,seller_id,seller_city,total_revenue,revenue_rank
0,4869f7a5dfa277a7dca6462dcf3b52b2,guariba,229472.63,1
1,53243585a1d6dc2643021fd1853d8905,lauro de freitas,222776.05,2
2,4a3ca9315b744ce9f8e9374361493884,ibitinga,200472.92,3
3,fa1c13f2614d7b5c4749cbc52fecda94,sumare,194042.03,4
4,7c67e1448b00f6e969d365cea6b010ab,itaquaquecetuba,187923.89,5


# 2 – Calculate year-over-year revenue growth using a CTE:

# Step-by-Step Logic:

| **Step** | **Action**                             | **Purpose**                                                |
| -------- | -------------------------------------- | ---------------------------------------------------------- |
| 1️⃣      | Identify raw data tables               | e.g., `order`, `order_items`, `sellers`, `products`                 |
| 2️⃣      | Write a CTE to calculate total revenue | `WITH seller_revenue AS (...)`                             |
| 3️⃣      | Inside the CTE: GROUP BY seller\_id    | Use `SUM(price)` to get total revenue per seller           |
| 4️⃣      | In the main query: SELECT from the CTE | Pull data from `seller_revenue`                            |
| 5️⃣      | Apply `RANK()` or `ROW_NUMBER()`       | Use a **window function** to rank sellers by total revenue |
| 6️⃣      | (Optional) Filter or format output     | Add conditions, sort, or limit rows (e.g., Top 5 sellers)  |


In [2]:
import pandas as pd
from sqlalchemy import create_engine

# SQL Server connection
engine = create_engine(
    "mssql+pyodbc://NARENDRA\\SQLEXPRESS/sql_to_python?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

# Query using CTE to calculate revenue by year and YoY growth
query = """
WITH yearly_revenue AS (
    SELECT 
        YEAR(order_purchase_timestamp) AS revenue_year,
        SUM(price) AS total_revenue
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    GROUP BY YEAR(order_purchase_timestamp)
)
SELECT 
    revenue_year,
    total_revenue,
    LAG(total_revenue) OVER (ORDER BY revenue_year) AS previous_year_revenue,
    ROUND(
        100.0 * (total_revenue - LAG(total_revenue) OVER (ORDER BY revenue_year)) / 
        NULLIF(LAG(total_revenue) OVER (ORDER BY revenue_year), 0), 2
    ) AS yoy_growth_percentage
FROM yearly_revenue;
"""

df = pd.read_sql(query, engine)
df.head()

# % = (total_revenue - LAG(total_revenue) OVER (ORDER BY revenue_year)) / LAG(total_revenue) OVER (ORDER BY revenue_year)
# NULLIF(..., 0)	 Prevents division by 0 — if last year's revenue is 0, it returns NULL.

,revenue_year,total_revenue,previous_year_revenue,yoy_growth_percentage
0,2016,4.978592e+04,NaN,NaN
1,2017,6.155807e+06,4.978592e+04,12264.55
2,2018,7.386051e+06,6.155807e+06,19.99


# 3. ✅ Question: Compare customer’s current vs. last order date
Goal: Find the time gap between each customer's orders


In [3]:
import pandas as pd
from sqlalchemy import create_engine

# SQL Server connection
engine = create_engine(
    "mssql+pyodbc://NARENDRA\\SQLEXPRESS/sql_to_python?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)

# Query: Compare current and last order dates per customer
query = """
SELECT 
    customer_id,
    order_id,
    order_purchase_timestamp,
    LAG(order_purchase_timestamp) OVER (PARTITION BY customer_id ORDER BY order_purchase_timestamp) AS last_order_date
FROM orders
ORDER BY customer_id, order_purchase_timestamp
"""

# Load result into DataFrame
df = pd.read_sql(query, engine)
df.head()

,customer_id,order_id,order_purchase_timestamp,last_order_date
0,00012a2ce6f8dcda20d059ce98491703,5f79b5b0931d63f1a42989eb65b9da6e,2017-11-14 16:08:26,None
1,000161a058600d5901f007fab4c27140,a44895d095d7e0702b6a162fa2dbeced,2017-07-16 09:40:32,None
2,0001fd6190edaaf884bcaf3d49edf079,316a104623542e4d75189bb372bc5f8d,2017-02-28 11:06:43,None
3,0002414f95344307404f0ace7a26f1d5,5825ce2e88d5346438686b0bba99e5ee,2017-08-16 13:09:20,None
4,000379cdec625522490c315e70c7a9fb,0ab7fb08086d4af9141453c91878ed7a,2018-04-02 13:42:17,None


# PARTITION BY vs ORDER BY:

| **Use**                               | **Explanation**                                                                 |
| ------------------------------------- | ------------------------------------------------------------------------------- |
| To **restart a window function**      | for each group of rows (using `PARTITION BY`)                                   |
| Common in `LAG()`, `LEAD()`, `RANK()` | when you want to calculate values **per group**, not across the entire table    |
| To **define row order** in group      | use `ORDER BY` inside the window function to **compare or rank rows correctly** |


#  When to use PARTITION BY?


| Case                               | Use `PARTITION BY`? | Why?                                  |
| ---------------------------------- | ------------------- | ------------------------------------- |
| Comparing years overall            | ❌ No                | No groups needed, just sort by year   |
| Comparing orders per customer      | ✅ Yes               | Restart window for each `customer_id` |
| Ranking products within categories | ✅ Yes               | Restart window per `product category` |


# Window Functions in SQL — With Simple Examples

| **Tool**          | **What It Does**                                                          | **Example Use Case**                         | **When to Use**                                   |
| ----------------- | ------------------------------------------------------------------------- | -------------------------------------------- | ------------------------------------------------- |
| `ROW_NUMBER()`    | Assigns a unique row number (1, 2, 3…) to each row in a group             | Top 1 product per category                   | When ranking rows                                 |
| `RANK()`          | Ranks with gaps for ties → e.g., 1, 1, 3                                  | Rank sellers by revenue                      | When handling tied values                         |
| `DENSE_RANK()`    | Ranks without gaps → e.g., 1, 1, 2                                        | Rank cities by number of customers           | When no rank gaps are needed                      |
| `LAG()`           | Shows previous row's value → e.g., 100, 90, 100 → LAG = NULL, 100, 90     | Compare current and previous purchase amount | When comparing previous row values                |
| `LEAD()`          | Shows next row's value → e.g., 100, 90, 100 → LEAD = 90, 100, NULL        | Compare current order with next order        | When comparing next row values                    |
| `AVG() OVER(...)` | Shows rolling average per row → e.g., 100, 200, 300 → AVG = 100, 150, 200 | Moving average of sales over time            | When keeping all rows but adding rolling averages |




# LAG(purchase_amount) OVER (ORDER BY order_id)

| Order ID | Purchase Amount | LAG Value |
| -------- | --------------- | --------- |
| 1        | 100             | `NULL`    |
| 2        | 90              | 100       |
| 3        | 100             | 90        |

Why?

Row 1 has no previous row → NULL

Row 2 looks back → sees 100

Row 3 looks back → sees 90


# LEAD(purchase_amount) OVER (ORDER BY order_id)

| Order ID | Purchase Amount | LEAD Value |
| -------- | --------------- | ---------- |
| 1        | 100             | 90         |
| 2        | 90              | 100        |
| 3        | 100             | `NULL`     |

Why?

Row 1 looks forward → sees 90

Row 2 looks forward → sees 100

Row 3 has no next row → NULL


# AVG() OVER (ORDER BY ...)

| Order ID | Amount | Rolling\_Avg |
| -------- | ------ | ------------ |
| 1        | 100    | 100          |
| 2        | 200    | 150          |
| 3        | 300    | 200          |

What’s Happening:

Row 1 → Avg(100) = 100

Row 2 → Avg(100, 200) = 150

Row 3 → Avg(100, 200, 300) = 200

# Find customers who ordered again within 6 months of first purchase

 Logic:

MIN(order_purchase_timestamp) → First purchase.

Second order (reorder) found using CASE logic.

DATEDIFF → Days between first and second order.

CASE → Classify whether it's within 6 months or not.

In [4]:
import pandas as pd
from sqlalchemy import create_engine

# Setup connection
engine = create_engine("mssql+pyodbc://NARENDRA\\SQLEXPRESS/sql_to_python?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

# SQL query
query = """
WITH first_orders AS (
    SELECT 
        customer_id,
        MIN(order_purchase_timestamp) AS first_order_date
    FROM orders
    GROUP BY customer_id
),
reorders AS (
    SELECT 
        o.customer_id,
        o.order_id,
        o.order_purchase_timestamp,
        fo.first_order_date,
        DATEDIFF(DAY, fo.first_order_date, o.order_purchase_timestamp) AS days_since_first
    FROM orders o
    JOIN first_orders fo ON o.customer_id = fo.customer_id
    WHERE o.order_purchase_timestamp > fo.first_order_date
)

SELECT 
    customer_id,
    COUNT(*) AS num_reorders_within_6_months
FROM reorders
WHERE days_since_first <= 180
GROUP BY customer_id
ORDER BY num_reorders_within_6_months DESC
"""

# Read into DataFrame
df = pd.read_sql(query, engine)
df.head()

,customer_id,num_reorders_within_6_months


# UNION, EXCEPT, INTERSECT — Identify Repeat Customers Across Years

In [4]:
# INTERSECT: Customers in both 2017 and 2018
query_intersect = """
SELECT TOP 100 customer_id
FROM orders
WHERE YEAR(order_purchase_timestamp) = 2017

INTERSECT

SELECT TOP 100 customer_id
FROM orders
WHERE YEAR(order_purchase_timestamp) = 2018
"""

df_intersect = pd.read_sql(query_intersect, engine)
df_intersect.head()

,customer_id


# ✅ UNION — Customers who ordered in 2017 or 2018 (no duplicates)

In [3]:
import pandas as pd
from sqlalchemy import create_engine

# 1. Connect to SQL Server
engine = create_engine("mssql+pyodbc://NARENDRA\\SQLEXPRESS/sql_to_python?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

# 2. Optimized UNION ALL query with filter + TOP (Quick Fixes)
query_union_optimized = """
SELECT TOP 100 customer_id
FROM orders
WHERE YEAR(order_purchase_timestamp) = 2017

UNION ALL

SELECT TOP 100 customer_id
FROM orders
WHERE YEAR(order_purchase_timestamp) = 2018
"""

# 3. Load limited test output
df_union_test = pd.read_sql(query_union_optimized, engine)
df_union_test.head()

,customer_id
0,9ef432eb6251297304e76186b10a928d
1,f88197465ea7920adcdbec7375364d82
2,503740e9ca751ccdda7ba28e9ab8f608
3,ed0271e0b7da060a393796590e7b737a
4,9bdf08b4b3b52b5526ff42d37d47f222


# ✅ EXCEPT — Customers who ordered in 2017 but NOT in 2018

In [5]:
import pandas as pd
from sqlalchemy import create_engine

# Connect to SQL Server
engine = create_engine("mssql+pyodbc://NARENDRA\\SQLEXPRESS/sql_to_python?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

# EXCEPT: Customers in 2017 but NOT in 2018
query_except = """
SELECT TOP 100 customer_id
FROM orders
WHERE YEAR(order_purchase_timestamp) = 2017

EXCEPT

SELECT TOP 100 customer_id
FROM orders
WHERE YEAR(order_purchase_timestamp) = 2018
"""

df_except = pd.read_sql(query_except, engine)
df_except.head()

,customer_id
0,059f7fc5719c7da6cbafe370971a8d70
1,05e996469a2bf9559c7122b87e156724
2,0a11cb0fb65032da800b780afcc1a1b7
3,0bf19317b1830a69e55b40710576aa7a
4,0f5b40f9925b49675464fb74869c6547
